In [ ]:
!git clone https://github.com/ccardenas93/lagtime.git

In [ ]:
#import shutil
#shutil.rmtree("/kaggle/working/lagtime")

In [ ]:
import sys
sys.path.insert(0, "/kaggle/working/lagtime/FCS")

import fcs_functions as fcs



In [ ]:
pip install python-docx

In [ ]:
import numpy as np
import pandas as pd
import sys

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
import os

## font and color for ploting
font = {'family': 'serif', 'color':  'darkred', 'weight': 'normal', 'size': 14}
color = [(174, 199, 232), (214, 39, 40)]  

for i in range(len(color)):    
    r, g, b = color[i]    
    color[i] = (r / 255., g / 255., b / 255.)  

In [ ]:
## Path of the gauges data (raingauges, surface and wastewater flow guages)
##path to the csv data file
inputfile1 = r'/kaggle/working/lagtime/FCS_input_files/9modeledStns.csv'
##path to the csv file with station information
path_sw = r'/kaggle/working/lagtime/FCS_input_files/wastewatercollectorslocation_new.csv'

## dataframe from the CVS data file
df2 = pd.read_csv(inputfile1, sep=',', decimal='.', index_col=[0], parse_dates=True)
df2 = df2.sort_index(axis=1)


# Read data about the guaging stations (ID, name, and x, y coordinates)
path_rain = r'/kaggle/working/lagtime/FCS_input_files/Rainfall_guageing_stations.csv'
Rainguages = fcs.station_data(path_rain)
SWguages = fcs.station_data(path_sw)

In [ ]:
##Create output folder

outfolder=r'/kaggle/working/lagtime/Modelling_result'
if not os.path.exists(outfolder):
    os.makedirs(outfolder)

In [ ]:
## Create list of Rainguages IDs
rain = []
for rg in Rainguages:
    rain.append(rg.id)
    

## Creat list of Surfacwwater flow uages IDs
flow_st = []
for st in SWguages:
    flow_st.append(st.id)


All_stn_results = pd.DataFrame()
# Lis of stations to be modelled
flow =  ['C01', 'C02', 'C11', 'U05', 'U06', 'U09', 'U11', 'U17', 'U19'] 
# flow =  ['C01' ]

# lag_times = {'C01': 12,
#              'C02': 15,
#              'C11': 9,
#              'U05': 15,
#              'U06': 15,
#              'U09': 15,
#              'U11': 9,
#              'U17': 18,
#              'U19': 18}
      
names = []
rsquared = []
WL = ['U06','C02','U09', 'C081','C082', 'C981','C982'] ## list of station of Water level
n = 5 ## number of RF stations closest to the flow station considered


In [ ]:
## Create dataframe of the rainfall of the n number of closest stations
df = pd.DataFrame()
for f in flow:
    print('station', f)
    
    if (f == st.id for st in SWguages):
        df_list = fcs.closest_stations(Rainguages, SWguages, f,n)
        #print('the flow station and the 10 closest RF stations are ' , df_list)
        df_list = [x for x in df_list if x  in df2.columns.values]   #df2 is the dataframe containing the data read from the CSV
        df = df2[df_list].copy() # the final dataframe for the DDM
        names.append(f)
    else:
          print('No {0} in flow stations list'.format(f))  
    
    # ## if there are missing values, fill them with the mean of the series
    # for col in df.columns.values:
    #     df[col].fillna(df[col].mean(), inplace=True)
    #     #df[col][df[col]<0] = df[col].mean()
        
    # abstract the data of the required length (here cosider only from 2016 to 2018)
    df = df.loc['2016-01-01':'2018-12-31']
   
    
 #%%   
    # get list of Rainfall stations associated with the flow station
    rfst =  list(df.columns.values)
    del rfst[0]  # the first in the list id the flow station
    nshift = 24  ## the lag time in 5 minutes interval (2hr = 24, 1hr = 12) 
    rg_names = [] ## list of correlated rf gauging stations name
    
    corrST = []
    maxcorr = []
    indmaxcorr = [] ##index of the correlated RF station  
    corrRG = [] 
    RF_back = 12    ## to increase the RF rolling sum duration
    rolling_sum_window = nshift-RF_back
    rolling_sum_windows =[3,6,9,12,15,18,21,24]
    stn_result = pd.DataFrame()
    for rolling_sum_window in rolling_sum_windows:
        for rg in rfst:
            #df[rg] = np.around(df[rg].rolling(nshift).apply(np.sum), decimals=2)
            rg_lag = rg+'_rs'+str(rolling_sum_window)
            # df[rg_lag] = df[rg].rolling(nshift).sum()
            df[rg_lag] = np.around(df[rg].rolling(rolling_sum_window).apply(np.sum, raw=True), decimals=2)
            corr = df[f].corr(df[rg_lag])
            corrRG.append(corr)
            rg_names.append(rg_lag)
            sw2rg_corr = dict(zip(rg_names, corrRG))
            
        sorted_corr_list = sorted(sw2rg_corr, key=sw2rg_corr.get, reverse=True)
    
        # sorted_rg = [s.replace('_lag', '') for s in sorted_corr_list]
        # sorted_rg=sorted_corr_list
        nrfs2c = 5  ## number of RF stations to consider    
        # df_rg = df[sorted_rg[:nrfs2c]]
        df_rg = df[sorted_corr_list[:nrfs2c]]
        if(df_rg.isnull().values.any()):
            print('there is nan in the rainfall data')
        
    #%%
         
        # lag the flow series by nshift ( the duration of the forecast time)   
        df_lag = pd.DataFrame()
        for i in range(10):
            #df[rg] = np.around(df[rg].rolling(nshift).apply(np.sum), decimals=2)
            lag = 'lag_'+str(i)
            name = f+'_lag_'+str(nshift+i)
            df_lag[name] = df[f].shift(nshift+i)
    
        
        df4 = pd.concat([df[f], df_lag], axis=1)
        if(df4.isnull().values.any()):
            print('there is nan in df[f]')
    
        if(not f in WL):
            df4 = df4[df4>=0]
    
            
        # add the 5 most correlated rainfall stations to the flow data frame
        df4 = pd.concat([df4, df_rg], axis=1)
            
        if(df4.isnull().values.any()):
            print('there is nan in df4')
            
        # drop the nans
        df4 = df4.dropna()
        
        print('Information of the dataframe for the modelling:/n')
        print(df4.info())
        
        # prepare the data for DDM
        xcols =  list(df4.columns.values)
        del xcols[0]   
        indexData = df4.index.values
        X = df4[xcols] #.values
        y = df4[f] #.values
        
    
        ## Training data size
        splits = TimeSeriesSplit(4) ## 3/4 for training and 1/4 for testing
        
        for trainIdx, testIdx in splits.split(X):
            trainIndex = trainIdx
            testIndex = testIdx
           
    
        X_train = X[:len(trainIndex)]
        X_test = X[len(trainIndex): (len(trainIndex)+len(testIndex))]
       
        y_train = y[:len(trainIndex)]
        y_test = y[len(trainIndex): (len(trainIndex)+len(testIndex))]
       
        print('Observations: %d' % (len(X_train)+len(X_test)))
        print('Training Observations: %d' % len(X_train))
        print('Testing Observations: %d' % len(X_test))
    
          ## normilize the input features      
        minMaxScaler = MinMaxScaler(feature_range=(0, 1))
        # X_train[X_train.columns] = minMaxScaler.fit_transform(X_train)
        # X_test[X_test.columns] = minMaxScaler.transform(X_test)
        X_train = minMaxScaler.fit_transform(X_train)
        X_test = minMaxScaler.transform(X_test)
        
        
        
        # ## transforming the data to better represent peaks
        # # standardizing
        # X_train, mean, std =fcs.standardize_fit(X_train)
        # X_test = fcs.standardize(X_test, mean, std)
        
        # # normalizing
        # X_train, xmax, xmin =fcs.normalize_fit(X_train)
        # X_test = fcs.normalize(X_test, xmax, xmin)
        
        
        
    #    #Wilson_Hilferty_transformation_fit
    #    X_train, mean, std = Wilson_Hilferty_transformation_fit(X_train)
    #    X_test = Wilson_Hilferty_transformation(X_test, mean, std)
        
        
        
        ## Lists to hold the evaluation criterial and model
        results = []
        models = []
        
        ## train the modl
        predictions_train, predictions_test, RF \
            = fcs.fit_RF(X_train, X_test, y_train, y_test)
            
        # # save the trained model
        # fcs.pickle_out(RF)
        
        ## create dataframes of training and testing resulsts for ploting
        pred_train = pd.Series(predictions_train, y_train.index)
        df_train = pd.concat([y_train, pred_train], axis=1)
        df_train.columns =['observed', 'estimated']
        
        pred_test = pd.Series(predictions_test, y_test.index)
        df_test = pd.concat([y_test, pred_test], axis=1)
        df_test.columns =['observed', 'estimated']
        
        ## evaluate the training and test of the model    
        r2_train, rmse_train, nse_train = fcs.evaluation_stat(y_train,predictions_train)
        r2_test, rmse_test, nse_test = fcs.evaluation_stat(y_test,predictions_test)
          
        ## Plot the resulsts
        
        
        # df_train = df_train.loc['2016-01-01':'2016-01-02']
        gaps = 1 ## to plt the data with gaps where there are NANs
        fig3 = fcs.plot_regression_result('RandomForest_Training', f,df_train,r2_train,color,gaps)
        fig4 = fcs.plot_regression_result('RandomForest_Test',f,df_test,r2_test,color,gaps)
            
        score = ['RF',r2_train, r2_test, rmse_train, rmse_test,nse_train, nse_test]
        results.append(score)
            
        models.append(RF)
        plots_list = [fig3, fig4]
        
        ##create resultas dataframe
        columnNames = ['Method','Rsqured_train','Rsquared_test','rmse_train','rmse_test','NSE_train', 'NSE_test']
        result = pd.DataFrame(results, columns=columnNames)
        
        ## save the results, figures and model in a word file
        input_list = X.columns.values
        input_train_len, input_features_len = X_train.shape
        input_test_len,_ = X_test.shape
        lag = nshift*5  ## lag time in minutes
    
        fcs.wirte_results_summary_to_word2(f,lag,input_train_len, input_features_len,
                                           input_list, input_test_len, 
                                           plots_list,result, models,rolling_sum_window,outfolder
                                           )
        
        score_stn = [f+'_'+str(rolling_sum_window),r2_train, r2_test, rmse_train, rmse_test,nse_train, nse_test]
        columnNames_stn = ['Station','Rsqured_train','Rsquared_test','rmse_train','rmse_test','NSE_train', 'NSE_test']
        results_stn = []
        results_stn.append(score_stn)
        result_stn_df = pd.DataFrame(results_stn, columns=columnNames_stn)
        stn_result = pd.concat([stn_result, result_stn_df],axis=0)
        All_stn_results = pd.concat([All_stn_results, result_stn_df],axis=0)
    fcs.wirte_all_stn_results(stn_result,outfolder,f)
# All_stn_results.sort_values("Station", inplace=True)
fcs.wirte_all_stn_results(All_stn_results,outfolder,'All stations with different rolling sum window')
outcsv = r'/kaggle/working/lagtime/Modelling_resultU05_different_Rollingsum sindwo.csv'
with open(outcsv,'w') as outfile:
        All_stn_results.to_csv(outfile, float_format='%.3f', sep=';',line_terminator='/n')
        
df_stn = {}      
lag_times = {}
for f in flow: 
    dfname = f+'df'
    df_stn[dfname] = All_stn_results[All_stn_results['Station'].str.contains(f)]
    df_stn[dfname].sort_values(['Rsquared_test', 'Rsqured_train'], ascending=[False, False],inplace=True)
    lag_times[f] = int(df_stn[dfname]['Station'].iloc[0].replace(f+'_', ''))
